# Benchmarks for SDE with quadratic non-linearity and additive noise

Benchmark simulations for Fig. 8 of [The two-particle irreducible effective action for classical stochastic processes](https://doi.org/10.1088/1751-8121/ac73c6).

In [ ]:
using DrWatson
@quickactivate "The Two-Particle Irreducible Effective Action for Classical Stochastic Processes"

using DifferentialEquations

In [ ]:
# parameters
α = 1.0
D = 1.5
β = 0.15

In [ ]:
# time parameters
T = 10.
n = 2^10
delta_t = T/n
times = abs(α) .* range(0, length=n + 1, stop=T) |> collect;

# initial conditions
x₀ = -10 * ones(ComplexF64, 1, 1)
F₀ = 2 * ones(ComplexF64, 1, 1);

In [ ]:
f(x, p, t) = α * x + β * x^2
g(x, p, t) = sqrt(D)
prob = SDEProblem(f, g, x₀[1] |> real, (0.0, T));

In [ ]:
# algo = SRIW1()
algo = SRIW2()

ensembleSize = 10
batch = "1";

## Single trajectory

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

In [ ]:
test_sol = solve(prob, algo);

figure(figsize=(4, 2.5))
plot(test_sol.t, test_sol.u)

xlim(0, T)
ylim(x₀[1] |> real, 0)
xlabel("\$ \\alpha t \$")
ylabel("\$ x(t) \$")
tight_layout()

## Ensemble average

In [ ]:
using Distributions

In [ ]:
# sample initial conditions
function prob_func(prob, i, repeat)
    u0=(prob.u0 + rand(Normal(0, F₀[1, 1] |> real |> sqrt)))
    remake(prob, u0=u0)
end;

In [ ]:
sim = solve(EnsembleProblem(prob, prob_func=prob_func), algo, EnsembleSerial(), trajectories=Int(ensembleSize));

In [ ]:
filter!(u -> (u.retcode |> String == "Success"), sim.u);

In [ ]:
x_sim, F_sim = EnsembleAnalysis.timepoint_meanvar(sim, times)

In [ ]:
# using HDF5
# h5write("../data/test.h5", ((algo |> typeof |> string) * "_" * string(ensembleSize))[1:end-2] * "_" * "batch_" * batch * "/x_sim", x_sim)
# h5write("../data/test.h5", ((algo |> typeof |> string) * "_" * string(ensembleSize))[1:end-2] * "_" * "batch_" * batch * "/F_sim", F_sim)